<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

**Import**

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

**Read data**

In [2]:
df = pd.read_csv('data/insurance.csv')

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.rename(columns={'Пол':'sex','Возраст':'age','Зарплата':'salary','Члены семьи':'family_members','Страховые выплаты':'insurance_payments'}, inplace = True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sex                 5000 non-null   int64  
 1   age                 5000 non-null   float64
 2   salary              5000 non-null   float64
 3   family_members      5000 non-null   int64  
 4   insurance_payments  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [6]:
df.isna().sum().sum()

0

In [7]:
df.duplicated().sum()

153

Удалим дубликаты записей, т.к. вероятность полного совпадения записей крайне мала

In [8]:
df.drop_duplicates(inplace = True)

In [9]:
df.duplicated().sum()

0

In [10]:
df.describe()

,sex,age,salary,family_members,insurance_payments
count,4847.000000,4847.000000,4847.000000,4847.000000,4847.000000
mean,0.498453,31.023932,39895.811842,1.203425,0.152259
std,0.500049,8.487995,9972.953985,1.098664,0.468934
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33200.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


## Умножение матриц

Предсказания:

$$
a = Xw
$$
$$
a'  = XPw'
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$
$$
w'= \arg\min_w MSE(XPw', y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Формула обучения преобразованной матрицы:

$$
w' = ((XP)^T) (XP))^{-1} (XP)^T y
$$

**Ответ:** Умножение матрицы признаков на случайную обратимую матрицу не изменит качество линейной регрессии

**Обоснование:**  докажем, что $$ a = a_A $$

  $$ w' = ((XP)^T) (XP))^{-1} (XP)^T y = (P^TX^TXP)^{-1}P^TX^Ty=(X^TXP)^{-1}(P^T)^{-1}P^TX^Ty=P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty = P^{-1}(X^TX)^{-1}X^Ty = P^{-1}w $$ 
  Отсюда $$a'= XPw' = XPP^{-1}w = Xw = a $$

**Проверим это на практике**

Обучим модель линейной модели изначальных данных

In [11]:
features = df.drop('insurance_payments', axis=1)
target = df['insurance_payments']

In [12]:
features.shape[0]

4847

In [13]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.4302010044852067


In [14]:
np.random.seed(0)
P = np.random.random((features.shape[1], features.shape[1]))
if np.linalg.det(P):
    features_p = features @ P
else:
    print('Матрица не обратима')

In [15]:
model.fit(features_p, target)
predictions = model.predict(features_p)
print(r2_score(target, predictions))

0.4302010044708513


**ВЫВОД** Как и следовало ожидать умножение матрицы признаков на случайную обратимую матрицу не изменило качество линейной регрессии, значеня r2 в обоих случаях равны

## Алгоритм преобразования

**Алгоритм**

Напишем функцию шифрования персональных данных

In [16]:
def encode(target_column,seed):
    np.random.seed(0)
    features = df.drop(target_column, axis=1)
    P = np.random.random((features.shape[1], features.shape[1]))
    if np.linalg.det(P):
        features_p = features @ P
        return features_p
    else:
        print('Матрица не обратима')

**Обоснование**

докажем, что $$ a = a_A $$

  $$ w' = ((XP)^T) (XP))^{-1} (XP)^T y = (P^TX^TXP)^{-1}P^TX^Ty=(X^TXP)^{-1}(P^T)^{-1}P^TX^Ty=P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty = P^{-1}(X^TX)^{-1}X^Ty = P^{-1}w $$ 
  Отсюда $$a'= XPw' = XPP^{-1}w = Xw = a $$

## Проверка алгоритма

In [18]:
encode('insurance_payments')

,0,1,2,3
0,47816.159626,19046.821778,39288.176764,26270.382725
1,36639.241064,14601.414441,30105.751495,20139.115638
2,20249.203960,8071.002825,16638.915829,11132.654732
3,40194.769953,16004.926305,33024.265492,22073.819645
4,25164.009197,10026.623866,20676.878699,13829.671933
...,...,...,...,...
4995,34415.758973,13708.798451,28276.978374,18906.692538
4996,50510.900958,20115.221583,41501.340997,27744.501206
4997,32677.776766,13013.436564,26848.372607,17947.547498
4998,31523.345621,12556.239315,25899.851536,17315.289153


Данные закодированы

In [20]:
model.fit(encode('insurance_payments'), target)
predictions_A = model.predict(encode('insurance_payments'))
print('r2:',r2_score(target, predictions))
print(r2_score(target, predictions)==r2_score(target, predictions_A))

r2: 0.4302010044708513
True


Значение метрики r2 до и после кодирования равны

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования